# VacationPy
---
## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,17.98,89,30,3.65,MG,1687493820
1,1,taksimo,56.3444,114.8794,28.74,34,45,1.50,RU,1687493820
2,2,port-aux-francais,-49.3500,70.2167,8.12,97,100,17.42,TF,1687493820
3,3,waitangi,-43.9535,-176.5597,11.01,81,8,1.79,NZ,1687493820
4,4,adamstown,-25.0660,-130.1015,20.63,50,45,8.60,PN,1687493820


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Max temperature less than or equal to 28 degrees but greater than or equal to 21
# Wind speed less than 4 m/s
# Zero cloudiness

ideal_weather_data_df = city_data_df.loc[(city_data_df["Max Temp"]>=21) & (city_data_df["Max Temp"]<=28) & 
                                         (city_data_df["Wind Speed"]<=4) & (city_data_df["Cloudiness"]==0), :]

# Drop any rows with null values
ideal_weather_data_df = ideal_weather_data_df.dropna()

# Display sample data
ideal_weather_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,67,at taj,24.2049,23.2857,24.23,26,0,3.00,LY,1687493831
91,91,great bend,38.3645,-98.7648,23.91,78,0,1.54,US,1687493834
102,102,jasper,37.2001,-94.3502,22.03,78,0,0.00,US,1687493836
148,148,petropavlovsk-kamchatsky,53.0452,158.6483,25.33,36,0,4.00,RU,1687493841
161,161,asilah,35.4650,-6.0348,23.98,67,0,2.23,MA,1687493842
180,180,milas,37.3164,27.7839,24.71,46,0,3.09,TR,1687493677
194,194,gargalianoi,37.0667,21.6333,22.42,64,0,1.78,GR,1687493846
212,212,kyzylorda,44.8528,65.5092,26.01,22,0,3.00,KZ,1687493849
234,234,galliano,29.4422,-90.2992,25.00,94,0,2.57,US,1687493851
272,272,stephenville,32.2207,-98.2023,27.44,77,0,3.09,US,1687493857


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
67,at taj,LY,24.2049,23.2857,26,
91,great bend,US,38.3645,-98.7648,78,
102,jasper,US,37.2001,-94.3502,78,
148,petropavlovsk-kamchatsky,RU,53.0452,158.6483,36,
161,asilah,MA,35.4650,-6.0348,67,
180,milas,TR,37.3164,27.7839,46,
194,gargalianoi,GR,37.0667,21.6333,64,
212,kyzylorda,KZ,44.8528,65.5092,22,
234,galliano,US,29.4422,-90.2992,94,
272,stephenville,US,32.2207,-98.2023,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
cat = "accommodation.hotel"
limit = 10
params = {
    "limit": limit,
    "categories": cat,
    "apiKey": geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = hotel_df.loc[index, "Lat"]
    Long = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Long},{Lat},{radius}"
    params["bias"] = f"proximity:{Long},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
at taj - nearest hotel: No hotel found
great bend - nearest hotel: Best Western
jasper - nearest hotel: Econo Lodge
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
asilah - nearest hotel: Hotel Las Palmas
milas - nearest hotel: Eldehan Otel
gargalianoi - nearest hotel: Messinian Horizons
kyzylorda - nearest hotel: Нур
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
stephenville - nearest hotel: No hotel found
brookhaven - nearest hotel: The Inn aon Whitworth
mostaganem - nearest hotel: Hôtel Munatec
hassi messaoud - nearest hotel: AMC
qom - nearest hotel: لاله
tataouine - nearest hotel: No hotel found
el abiodh sidi cheikh - nearest hotel: No hotel found
mournies - nearest hotel: Oasis
awjilah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
67,at taj,LY,24.2049,23.2857,26,No hotel found
91,great bend,US,38.3645,-98.7648,78,Best Western
102,jasper,US,37.2001,-94.3502,78,Econo Lodge
148,petropavlovsk-kamchatsky,RU,53.0452,158.6483,36,ООО Постоялый двор
161,asilah,MA,35.4650,-6.0348,67,Hotel Las Palmas
180,milas,TR,37.3164,27.7839,46,Eldehan Otel
194,gargalianoi,GR,37.0667,21.6333,64,Messinian Horizons
212,kyzylorda,KZ,44.8528,65.5092,22,Нур
234,galliano,US,29.4422,-90.2992,94,Fairfield Inn & Suites by Marriott Cut Off-Gal...
272,stephenville,US,32.2207,-98.2023,77,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_humidity_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_humidity_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)